# 第二章第五节：结合前面所学的练习：一个有限脉冲响应（FIR）滤波器（Filter）

## 本节的目的

现在您已经学习了Chisel的基础知识，那么让我们利用这些知识来构建一个FIR（有限脉冲响应）滤波器模块！ FIR滤波器在数字信号处理中非常常见。此外，FIR滤波器将在第三章中频繁出现，因此最好不要跳过这一节！如果您不熟悉FIR滤波器，请访问[维基百科](https://en.wikipedia.org/wiki/Finite_impulse_response)来了解更多信息。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# FIR滤波器

您将要设计的这个FIR滤波器包含如下操作：

<img src="images/fir.jpg" width="720">

基本上，这将滤波器系数与输入信号的每个元素分别相乘，并输出相加之后的总和（也称为 _卷积_）。

或者，用信号来定义：

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ 是在时间 $n$ 的输出信号
 - $x[n]$ 是输入信号
 - $b_i$ 是滤波器系数或脉冲响应
 - $n-1$, $n-2$, ... 表示在时间 $n$ 分别被延迟 1, 2, ... 个时钟周期的下标
 
## 规格说明

构建一个4元FIR滤波器，其中四个滤波器的系数作为参数。我们为您提供模块代码的框架和基本测试。
请注意，输入和输出都是8位无符号整数。您需要使用类似移位寄存器的结构来保存必要的状态（如延迟的信号值）。并使用下面提供的测试来检查您的实现。
具有常量输入的寄存器可以使用移位值为1的`ShiftRegister`来创建或使用`RegNext`。

注意：要通过测试的话，寄存器初始值必须为`0.U`。

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

In [ ]:
// 可用性测试（sanity test）：具有全零系数的滤波器输出应始终为零
Driver(() => new My4ElementFir(0, 0, 0, 0)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 5)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 0)
  }
}

In [ ]:
// 简单的4点移动滤波器
Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

In [ ]:
// 非对称滤波器
Driver(() => new My4ElementFir(1, 2, 3, 4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1*1, 0*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 6)  // 4*1, 1*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 14)  // 3*1, 4*2, 1*3, 0*4
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 24)  // 2*1, 3*2, 4*3, 1*4
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 36)  // 7*1, 2*2, 3*3, 4*4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 32)  // 0*1, 7*2, 2*3, 3*4
  }
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in \* b0.U(8.W) + 
    x_n1 \* b1.U(8.W) +
    x_n2 \* b2.U(8.W) + 
    x_n3 \* b3.U(8.W)
</pre></article></div></section></div>

---
# FIR滤波器的生成器

在这一节中，我们将使用[本教程 3.2一节：生成器：容器（Collection）]中的示例稍加修改。
如果您尚未启动学习3.2节，请不要担心。
您将了解`MyManyDynamicElementVecFir`如何工作的详细信息，但基本思想是FIR滤波器的生成器。

生成器有一个参数：长度（length)。
该参数指定滤波器具有多少个抽头（tap），并且抽头（tap）是硬件模块`Module`的输入。

该生成器有3个输入：
* in，滤波器的输入值
* valid，一个布尔值，表示当前输入是否有效
* consts，所有抽头（tap）的向量

和1输出：
* out，滤波器的输出

<img src="images/fir.jpg" style="width:450px;"/>

In [7]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  // 非常紧凑的代码！您将在之后学习
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

defined class MyManyDynamicElementVecFir

---
# DspBlock

将DSP组件集成到更大的系统中可能是一个有挑战性的工作并且容易出错。
[dsptools github库的rocket部分]](https://github.com/ucb-bar/dsptools/tree/master/rocket) 包含有用的生成器来帮助完成这些任务。

其中一个核心抽象是`DspBlock`。
`DspBlock`具有：
* AXI-4流的输入和输出
* 具有内存映射的状态和控制信号（在本例中为AXI4）

<img src="images/fir_filter.png" style="width:800px;"/>

`DspBlock`使用来自rocket的diplomacy接口。
[这个网站](https://www.lowrisc.org/docs/diplomacy/) 对diplomacy有基本的介绍，但是现在不需要过于担心这个例子是如何工作的。
当您需要将许多不同的模块连在一起形成一个复杂的SoC时，diplomacy真的很出彩。
在这个例子中，我们只制作一个外设。
当diplomacy接口和`StandaloneBlock`特征（traits）合在一起使用的时候，可以使这个diplomacy接口成为顶层IO接口。
只有当`DspBlock`被用作顶层接口而没有任何其他diplomacy连接时，您才需要它们。

下面的代码将FIR滤波器包装在AXI4接口中。


In [11]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._
import freechips.rocketchip.regmapper._

//
// 基类 FIRBlock 
// 它可以被继承，从而生成具有 TileLink, AXI4, APB, AHB 等接口的FIR滤波器
//
abstract class FIRBlock[D, U, EO, EI, B <: Data](val nFilters: Int, val nTaps: Int)(implicit p: Parameters)

// HasCSR 表示将使用 RegMapper API 来定义具有内存接口的状态和控制寄存器
extends DspBlock[D, U, EO, EI, B] with HasCSR {
    // 用于流接口（streaming interface ）的diplomacy节点
    // identity node表示输出和输入的参数相同
    val streamNode = AXI4StreamIdentityNode()
    
    // 定义将要被展开（elaborate）的硬件
    lazy val module = new LazyModuleImp(this) {
        // 从diplomacy节点拿到流的输入输出接口的线
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)

        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        // 创建存储抽头（tap）的寄存器
        val taps = Reg(Vec(nFilters, Vec(nTaps, UInt(8.W))))

        // 为抽头（tap）创建内存地址映射，另外，第一个地址是只读的，用来指定滤波器通道数目
        val mmap = Seq(
            RegField.r(64, nFilters.U, RegFieldDesc("nFilters", "Number of filter lanes"))
        ) ++ taps.flatMap(_.map(t => RegField(8, t, RegFieldDesc("tap", "Tap"))))

        // 为该类生成内存接口的硬件
        // regmap是抽象的（未实现）。与 AXI4HasCSR 或 TLHasCSR 等混合的话，
        // 将使得 regmap 会被定义（为特定的接口）
        regmap(mmap.zipWithIndex.map({case (r, i) => i * 8 -> Seq(r)}): _*)

        // 生成 FIR 滤波器通道，并连接输入和抽头（tap）
        val outs = for (i <- 0 until nFilters) yield {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            fir.io.consts := taps(i)            
            fir.io.out
        }

        val output = if (outs.length == 1) {
            outs.head
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }

        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

// 创建具有 AXI4 接口的 FIRBlock
abstract class AXI4FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends FIRBlock[AXI4MasterPortParameters, AXI4SlavePortParameters, AXI4EdgeParameters, AXI4EdgeParameters, AXI4Bundle](nFilters, nTaps) with AXI4DspBlock with AXI4HasCSR {
    override val mem = Some(AXI4RegisterNode(
        AddressSet(0x0, 0xffffL), beatBytes = 8
    ))
}

// 运行下面的代码将显示所生成的 firrtl 代码
// 注意 LazyModules 不是真正的 Chisel 模块 - 在调用 Chisel 驱动时需要调用它的 “.module” 属性
// 还要注意必须要与 AXI4StandaloneBlock 混合在一起 - 如果您忘了它，您会得到奇怪的diplomacy错误，
// 因为内存接口需要 master 和流接口连接。 AXI4StandaloneBlock 将添加顶层的 IO 接口

// println(chisel3.Driver.emit(() => LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock).module))

import dspblocks._

import freechips.rocketchip.amba.axi4._

import freechips.rocketchip.amba.axi4stream._

import freechips.rocketchip.config._

import freechips.rocketchip.diplomacy._

import freechips.rocketchip.regmapper._

//
// 基类 FIRBlock 
// 它可以被继承，从而生成具有 TileLink, AXI4, APB, AHB 等接口的FIR滤波器
//

defined class FIRBlock
defined class AXI4FIRBlock

## 测试


测试 `DspBlock` 会稍微有点不同。
现在我们需要处理内存接口和`LazyModule`。
dsptools有一些功能可以帮助用来测试`DspBlock`。

一个重要的功能是`MemMasterModel`。
该特征（trait）定义了诸如`memReadWord`和`memWriteWord`的函数 - 用于生成内存流量的泛型函数。
这允许您编写一个可以为特定内存接口特化的通用测试 - 例如，您编写一个通用测试，然后为TileLink和AXI4接口特化它。

下面的代码以这种方式测试`FIRBlock`：

In [12]:
import dsptools.tester.MemMasterModel
import freechips.rocketchip.amba.axi4

abstract class FIRBlockTester[D, U, EO, EI, B <: Data](c: FIRBlock[D, U, EO, EI, B]) extends PeekPokeTester(c.module) with MemMasterModel {
    // 检查地址 0 是滤波器通道的数目
    require(memReadWord(0) == c.nFilters)
    // 将所有的抽头（tap）设置为 1
    for (i <- 0 until c.nFilters * c.nTaps) {
        memWriteWord(8 + i * 8, 1)
    }
}

// 为 AXI4 特化我们的测试
class AXI4FIRBlockTester(c: AXI4FIRBlock with AXI4StandaloneBlock) extends FIRBlockTester(c) with AXI4MasterModel {
    def memAXI = c.ioMem.get
}

// 在 lazymodule 上调用我们的测试看上去会有一点奇怪 
// 注意，firblocktester 需要一个 lazymodule 作为参数，而不是一个 module (例如在 "extends PeekPokeTester()" 中调用的).
val lm = LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock)
chisel3.iotesters.Driver(() => lm.module) { _ => new AXI4FIRBlockTester(lm) }

[info] [0.000] Elaborating design...
[deprecated] RegMapper.scala:160 (36 calls): litArg is deprecated: "litArg is deprecated, use litOption or litTo*Option"
[deprecated] ReduceOthers.scala:11 (100 calls): litArg is deprecated: "litArg is deprecated, use litOption or litTo*Option"
[deprecated] ReduceOthers.scala:27 (100 calls): litArg is deprecated: "litArg is deprecated, use litOption or litTo*Option"
[deprecated] MuxLiteral.scala:37 (192 calls): litArg is deprecated: "litArg is deprecated, use litOption or litTo*Option"
[warn] There were 4 deprecated function(s) used. These may stop compiling in a future release - you are encouraged to fix these issues.
[warn] Line numbers for deprecations reported by Chisel may be inaccurate; enable scalac compiler deprecation warnings via either of the following methods:
[warn]   In the sbt interactive console, enter:
[warn]     set scalacOptions in ThisBuild ++= Seq("-unchecked", "-deprecation")
[warn]   or, in your build.sbt, add the line:
[warn]

import dsptools.tester.MemMasterModel

import freechips.rocketchip.amba.axi4


defined class FIRBlockTester
defined class AXI4FIRBlockTester
lm: AXI4FIRBlock with AXI4StandaloneBlock = ammonite.$sess.cmd11$Helper$$anon$1@188d1f55
res11_5: Boolean = true

<span style="color:red">**练习：TileLink**</span><br>

添加一个使用TileLink作为其内存互连的`FIRBlock`版本，并扩展`FIRBlockTester`以使用TileLink。

---
# 本节结束!
​
[返回顶部](#top)